Применение Beggs_Brill_class, попытка исправления и приведения в боевую готовность

Импорт необходимых модулей

In [1]:
import sys
sys.path.append('../')
import uPVT.PVT as PVT
import Beggs_Brill_class as BB
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)



Создание класса для извлечения и хранения данных на каждом шаге расчета

In [2]:
class Data():
    def __init__(self):
        self.dicts = []  # список словарей атрибутов класса
        self.lists = []  # списки для хранения
        self.h_list = []
        self.p_list = []
        self.t_list = []
    def get_data(self, Beggs_Brill):
        """
        Получение атрибутов (всех данных) из класса
        """
        this_data = Beggs_Brill.__dict__
        self.dicts.append(this_data)
        for key, value in this_data.items():  # преобразование словаря в итерируемый список
            temp = [key,value]
            self.lists.append(temp)
    def clear_data(self): 
        """
        Очистка данных
        """
        self.dicts = []
        self.lists = []
        self.h_list = []
        self.p_list = []
        self.t_list = []
    def get_values(self, number):
        """
        Получение массива определенного параметра по его номеру
        """
        amounts_of_param_in_one_banch = 56
        amounts_of_banches = len(self.lists) / amounts_of_param_in_one_banch
        one_parametr = []
        k = number
        for i in range(int(amounts_of_banches)):
            one = self.lists[k]
            k = k + amounts_of_param_in_one_banch
            one_parametr.append(one)
        values = []
        for i in one_parametr:
            values.append(i[-1])    
        return values
    def get_name(self, number):
        """
        Получение названия параметра по его номеру
        """
        return self.lists[number][0]
        

Создание требуемых экземляров класса

In [3]:
fluid = PVT.FluidStanding()
hydr_cor = BB.Beggs_Brill_cor()
hydr_data = Data()


Задание начальных условий

In [4]:

p_initial_mpa = 7
t_initial_c = 90
h_initial_m = 3000
step_m = 10
step_cm = 0.03


PT = BB.PT(p_initial_mpa,t_initial_c)

hydr_cor.gas_rate_on_surface_m3day = hydr_cor.oil_rate_on_surface_m3day * fluid.rsb_m3m3

Списки для хранения P,T,H и обращения к ним при расчетах

In [5]:
h_list = [h_initial_m]
p_list = [p_initial_mpa]
t_list = [t_initial_c]


### Основной расчет распределения давления и температуры

Температура меняетс линейно

In [6]:
for i in range(300):
    if PT.p_mpa > 0:
        
        fluid.calc(PT.p_mpa * 10,PT.T_C)
        hydr_cor.print_all = False
        hydr_cor.mu_gas_pasec = fluid.mu_gas_cP /1000
        hydr_cor.mu_oil_pasec = fluid.mu_oil_cP /1000
        hydr_cor.rho_oil_kgm3 = fluid.rho_oil_kgm3
        hydr_cor.rho_gas_kgm3 = fluid.rho_gas_kgm3
        
        hydr_cor.rs_m3m3 = fluid.rs_m3m3
        hydr_cor.bg_m3m3 = fluid.bg_m3m3
        hydr_cor.bo_m3m3 = fluid.bo_m3m3
        grad_pam = hydr_cor.calc_grad(PT)
        
        p = p_list[-1] - step_m * grad_pam / 10**6
        t = t_list[-1] - step_m * step_cm
        h = h_list[-1] - step_m
        
    if p > 0:
        p_list.append(p)
    else:
        p_list.append(p_list[-1])
        
    t_list.append(t)
    h_list.append(h)
    
    hydr_data.get_data(hydr_cor)
    hydr_data.h_list.append(h)
    hydr_data.t_list.append(t)
    hydr_data.p_list.append(p)
    PT.p_mpa = p
    PT.p_pa = p * 10**6
    PT.T_C = t
    hydr_cor.pressure_pa = p * 10**6
    hydr_cor.temperature_c = t
    

Пример получения имени параметра

In [7]:

hydr_data.get_name(0)

'mu_oil_pasec'

Список параметров

In [8]:
for i in range(56):
    print('Номер '+ str(i) + ' для получения параметра ' + hydr_data.get_name(i))

Номер 0 для получения параметра mu_oil_pasec
Номер 1 для получения параметра mu_gas_pasec
Номер 2 для получения параметра mu_water_pasec
Номер 3 для получения параметра mu_liquid_pas
Номер 4 для получения параметра sigma_kgsec2
Номер 5 для получения параметра watercut_percent
Номер 6 для получения параметра epsilon_friction_m
Номер 7 для получения параметра diametr_inner_m
Номер 8 для получения параметра oil_rate_on_surface_m3day
Номер 9 для получения параметра gas_rate_on_surface_m3day
Номер 10 для получения параметра water_rate_on_surface_m3day
Номер 11 для получения параметра Rp_m3m3
Номер 12 для получения параметра rs_m3m3
Номер 13 для получения параметра rsw_m3m3
Номер 14 для получения параметра rho_oil_kgm3
Номер 15 для получения параметра rho_water_kgm3
Номер 16 для получения параметра rho_liquid_kgm3
Номер 17 для получения параметра rho_gas_kgm3
Номер 18 для получения параметра bo_m3m3
Номер 19 для получения параметра bw_m3m3
Номер 20 для получения параметра bg_m3m3
Номер 21 дл

Функции для удобного построения графиков

In [9]:
def trace(number_param):
    tracep = go.Scattergl(
        x = hydr_data.get_values(number_param),
        y = hydr_data.h_list,
        name = hydr_data.get_name(number_param),
        mode = 'markers'
    )
    return tracep

def plot():
    layout = dict(title = 'БеггсБрилл снизу вверх, H(P,T)',  yaxis=dict(autorange='reversed'))

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

Построение графиков

In [10]:
tracep = go.Scattergl(
    x = hydr_data.p_list,
    y = hydr_data.h_list,
    name = 'P_MPa',
    mode = 'markers'
)

tracet = go.Scattergl(
    x = hydr_data.t_list,
    y = hydr_data.h_list,
    name = 'T_C',
    mode = 'markers'
)

data = [tracep, tracet]

plot()

In [11]:
trace1 = trace(0)
trace2 = trace(1)
trace3 = trace(2)
trace4 = trace(3)
trace5 = trace(40)
data = [trace1, trace2, trace3, trace4, trace5]

plot()

In [12]:
trace1 = trace(14)
trace2 = trace(15)
trace3 = trace(16)
trace4 = trace(17)
trace5 = trace(41)
trace6 = trace(48)

data = [trace1, trace2, trace3, trace4,  trace5,  trace6]

plot()

In [13]:
trace1 = trace(24)
trace2 = trace(25)
trace3 = trace(26)
trace4 = trace(27)

data = [trace1, trace2, trace3, trace4]

plot()

In [14]:
trace1 = trace(28)
trace2 = trace(30)

data = [trace1, trace2]

plot()

In [15]:
trace1 = trace(33)
trace2 = trace(37)
trace3 = trace(49)
trace4 = trace(53)
data = [trace1, trace2, trace3, trace4]

plot()


In [16]:
trace1 = trace(34)
trace2 = trace(35)
trace3 = trace(36)

data = [trace1, trace2, trace3]

plot()

In [17]:
trace1 = trace(43)

data = [trace1]

plot()

6